# Decision Tree Algorithm
Decision trees can be applied to both regression and classification problems.
It is one of the predictive modelling approaches used in statistics, data mining and machine learning. It uses a decision tree (as a predictive model) to go from observations about an item (represented in the branches) to conclusions about the item's target value (represented in the leaves). Tree models where the target variable can take a discrete set of values are called classification trees; in these tree structures, leaves represent class labels and branches represent conjunctions of features that lead to those class labels. Decision trees where the target variable can take continuous values (typically real numbers) are called regression trees. Decision trees are among the most popular machine learning algorithms given their intelligibility and simplicity.

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.py            plot as plt 
import seaborn as sns

%matplotlib inline

In [4]:
dummy_train_data = np.array([
    ['Green', 3, 'Apple'],
    ['Red', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
],dtype='O')
headers = ['color','diameter','fruit']

## CART(Classification and Regression Trees)

The representation for the CART model is a binary tree.

This is your binary tree from algorithms and data structures, nothing too fancy. Each root node represents a single input variable (x) and a split point on that variable (assuming the variable is numeric).

The leaf nodes of the tree contain an output variable (y) which is used to make a prediction.

Given a dataset with two inputs (x) of height in centimeters and weight in kilograms the output of sex as male or female, below is a crude example of a binary decision tree (completely fictitious for demonstration purposes only).


                                        Is diameter >= 3 ?
                                                 |
                                    _____________|_____________
                               True|                           |False
                                   |                           |
                          Is color == Yellow ?               Grape
                                 |
                    _____________|_____________
               True|                           |False
                   |                           |
                  Lemon                      Apple

## Question(Parition concept of the dataset)

- like a `Threshold`
- Question is a concept for split the node.
- It represents a True or False question. 
- match to see if it will go to which branch

In [5]:

class Question:
    def __init__(self, column_index, value, header):
        self.column_index = column_index
        self.value = value
        self.header = header

    def match(self, example):
        if isinstance(example, list):
            example = np.array(example, dtype="O")
        val = example[self.column_index]

        # adding numpy int and float data types as well
        if isinstance(val, (int, float, np.int64, np.float64)):
            # a condition for question to return True or False for numeric value
            return float(val) >= float(self.value)
        else:
            return str(val) == str(self.value)  # categorical data comparison

    def __repr__(self):
        condition = "=="
        if isinstance(self.value, (int, float, np.int64, np.float64)):
            condition = ">="
        return f"Is {self.header} {condition} {self.value} ?"

In [6]:
q1 = Question(column_index=0,value='apple',header='fruit')
q1

Is fruit == apple ?

In [7]:
q1.match(['apple',10])

True

In [8]:
q2 = Question(column_index=1,value=30,header='weight')
q2

Is weight >= 30 ?

In [9]:
q2.match(['apple',10])

False

## Partition rows

- partition rows in true side and false side using the Question we have already defined.

                                    ['color','diameter','fruit']
                                    
                                     [['Green', 3, 'Apple'],
                                     ['Red', 3, 'Apple'],
                                     ['Red', 1, 'Grape'],
                                     ['Red', 1, 'Grape'],
                                     ['Yellow', 3, 'Lemon']]
                                    
                                        Is color == Red ?
                                                 |
                                    _____________|_____________
                               True|                           |False
                                   |                           |
                          [['Red', 3, 'Apple'],          [['Green', 3, 'Apple'],
                          ['Red', 1, 'Grape'],           ['Yellow', 3, 'Lemon']]
                          ['Red', 1, 'Grape']]               

In [19]:
def partition(rows, question):
    true_idx, false_idx = [], []
    for idx, row in enumerate(rows):
        if question.match(row):
            true_idx.append(idx)
        else:
            false_idx.append(idx)
    return true_idx, false_idx

In [20]:
dummy_train_data

array([['Green', 3, 'Apple'],
       ['Red', 3, 'Apple'],
       ['Red', 1, 'Grape'],
       ['Red', 1, 'Grape'],
       ['Yellow', 3, 'Lemon']], dtype=object)

In [21]:
Question(column_index=0,value='Red',header=headers[0])

Is color == Red ?

In [22]:
left, right = partition(dummy_train_data,Question(column_index=0,value='Red',header=headers[0]))

In [25]:
dummy_train_data[left,:]

array([['Red', 3, 'Apple'],
       ['Red', 1, 'Grape'],
       ['Red', 1, 'Grape']], dtype=object)

In [26]:
dummy_train_data[right,:]

array([['Green', 3, 'Apple'],
       ['Yellow', 3, 'Lemon']], dtype=object)

## Classification Tree
## Gini Impurity

Used by the CART (classification and regression tree) algorithm for classification trees, Gini impurity is a measure of how often a randomly chosen element from the set would be incorrectly labeled if it was randomly labeled according to the distribution of labels in the subset. The Gini impurity can be computed by summing the probability $p_{i}$ of an item with label i being chosen times the probability 

$\sum_{k \neq i}{p_k} = 1 - {p_i}$

of a mistake in categorizing that item. It reaches its minimum (zero) when all cases in the node fall into a single target category.

$$
{I_G(p)} = {\sum_{i=1}^{J}{(p_i\sum_{k \neq i}p_k)}} = {\sum_{i=1}^{J}p_i(1 - p_i)} = {\sum_{i=1}^{J}(p_i - p_i^2)} = {\sum_{i=1}^{J}p_i - \sum_{i=1}^{J}p_i^2} = { 1 - \sum_{i=1}^{J}p_i^2} 
$$



In [28]:
def gini_impurity(arr):
    classes, counts = np.unique(arr, return_counts=True)
    gini_score = 1 - np.square(counts / arr.shape[0]).sum(axis=0)
    return gini_score

In [29]:
print(gini_impurity(np.array([0,0,0,0,0,0,0,1,1,1,1,1,1])))

0.4970414201183432


In [30]:
print(gini_impurity(np.array(['apple','apple','apple','apple'])))

0.0


In [31]:
print(gini_impurity(np.array(['apple','apple','apple','orange'])))

0.375


In [32]:
print(gini_impurity(np.array(['apple','apple','apple','orange','orange','orange'])))

0.5


## Entropy

$H(T) = I_E(P) = - {\Sigma_{i=1}^{J}{p_i}log_2{p_i}}$

In [34]:
def entropy(arr):
    classes, counts = np.unique(arr, return_counts=True)
    p = counts / arr.shape[0]
    entropy_score = (-p * np.log2(p)).sum(axis=0)
    return entropy_score

In [35]:
print(entropy(np.array([0,0,0,0,0,0,0,1,1,1,1,1,1])))

0.9957274520849255


In [36]:
print(entropy(np.array(['apple','apple','apple','apple'])))

0.0


In [37]:
print(entropy(np.array(['apple','apple','apple','orange'])))

0.8112781244591328


In [38]:
print(entropy(np.array(['apple','apple','apple','orange','orange','orange'])))

1.0


## Information Gain

Information gain is the reduction in entropy or surprise by transforming a dataset and is often used in training decision trees. Information gain is calculated by comparing the entropy of the dataset before and after a transformation.

Inforgation Gain = H(Before splitting) - H(After splitting)

In [72]:
def info_gain(left, right, parent_uncertainty):

    pr = left.shape[0] / (left.shape[0] + right.shape[0])

    child_uncertainty = pr * \
        gini_impurity(left) - (1 - pr) * gini_impurity(right)

    info_gain_value = parent_uncertainty - child_uncertainty
    return info_gain_value

In [73]:
headers,dummy_train_data

(['color', 'diameter', 'fruit'],
 array([['Green', 3, 'Apple'],
        ['Red', 3, 'Apple'],
        ['Red', 1, 'Grape'],
        ['Red', 1, 'Grape'],
        ['Yellow', 3, 'Lemon']], dtype=object))

In [74]:
curr_uncert = gini_impurity(dummy_train_data[:,-1])
curr_uncert

0.6399999999999999

In [75]:
ti , fi = partition(dummy_train_data,Question(0,'Green',headers[0]))

In [76]:
ts = dummy_train_data[ti,:]
ts

array([['Green', 3, 'Apple']], dtype=object)

In [77]:
fs = dummy_train_data[fi,:]
fs

array([['Red', 3, 'Apple'],
       ['Red', 1, 'Grape'],
       ['Red', 1, 'Grape'],
       ['Yellow', 3, 'Lemon']], dtype=object)

In [78]:
info_gain(ts[:,-1],fs[:,-1],curr_uncert)

1.14

## Find best split in all options

- based on the max information gain. find the best split 

In [87]:
def find_best_split(X, y):
    max_gain = -1
    best_split_question = None

    parent_uncertainty = gini_impurity(y)

    m_samples, n_labels = X.shape

    for col_index in range(n_labels):  # iterate over feature columns
        # get unique values from the feature
        unique_values = np.unique(X[:, col_index])
        for val in unique_values:  # check for every value and find maximum info gain

            ques = Question(
                column_index=col_index,
                value=val,
                header=headers[col_index]
            )

            t_idx, f_idx = partition(X, ques)
            # if it does not split the data
            # skip it
            if len(t_idx) == 0 or len(f_idx) == 0:
                continue

            true_y = y[t_idx, :]
            false_y = y[f_idx, :]

            gain = info_gain(true_y, false_y, parent_uncertainty)
            if gain > max_gain:
                max_gain, best_split_question = gain, ques

    return max_gain, best_split_question, parent_uncertainty          

In [88]:
find_best_split(dummy_train_data[:,:-1],dummy_train_data[:,[-1]])

(1.14, Is color == Green ?, 0.6399999999999999)

## Tree Node - Leaf node, Decision Node

In [89]:
class Node:
    def __init__(self, question=None, true_branch=None, false_branch=None, uncertainty=None, *, leaf_value=None):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        self.uncertainty = uncertainty
        self.leaf_value = leaf_value
    
    @property
    def _is_leaf_node(self):
        return self.leaf_value is not None


## Build Tree

- start recursion
  - find best split for the data (actually the question)
  - if gain == 0 then it is already a leaf node\
  - else get paritition based on the question's answer true_rows, false_rows 
  - build tree from true_rows and false_true and get Decision node with question, left and right branch

## Classification/ Prediction

- traverse the tree and use the question to match and classify

## Creating a Model

In [147]:
import numpy as np



class Node:
    def __init__(self, question=None, true_branch=None, false_branch=None, uncertainty=None, *, leaf_value=None):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        self.uncertainty = uncertainty
        self.leaf_value = leaf_value

    @property
    def _is_leaf_node(self):
        return self.leaf_value is not None


class Question:
    def __init__(self, column_index, value, header):
        self.column_index = column_index
        self.value = value
        self.header = header

    def match(self, example):
        if isinstance(example, list):
            example = np.array(example, dtype="O")
        val = example[self.column_index]

        # adding numpy int and float data types as well
        if isinstance(val, (int, float, np.int64, np.float64)):
            # a condition for question to return True or False for numeric value
            return float(val) >= float(self.value)
        else:
            return str(val) == str(self.value)  # categorical data comparison

    def __repr__(self):
        condition = "=="
        if isinstance(self.value, (int, float, np.int64, np.float64)):
            condition = ">="
        return f"Is {self.header} {condition} {self.value} ?"


class DecisionTreeClassifier:
    def __init__(self, max_depth=100, min_samples_split=2, criteria='gini'):
        self._X = None
        self._y = None
        self._feature_names = None
        self._target_name = None
        self._tree = None
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.criteria = criteria

    def _count_dict(self, a):
        unique_values = np.unique(a, return_counts=True)
        zipped = zip(*unique_values)
        return dict(zipped)

    def _gini_impurity(self, arr):
        classes, counts = np.unique(arr, return_counts=True)
        gini_score = 1 - np.square(counts / arr.shape[0]).sum(axis=0)
        return gini_score

    def _entropy(self, arr):
        classes, counts = np.unique(arr, return_counts=True)
        p = counts / arr.shape[0]
        entropy_score = (-p * np.log2(p)).sum(axis=0)
        return entropy_score

    def _partition(self, rows, question):
        true_idx, false_idx = [], []
        for idx, row in enumerate(rows):
            if question.match(row):
                true_idx.append(idx)
            else:
                false_idx.append(idx)
        return true_idx, false_idx

    def _info_gain(self, left, right, parent_uncertainty):

        pr = left.shape[0] / (left.shape[0] + right.shape[0])

        if self.criteria == "entropy":
            child_uncertainty = pr * \
                self._entropy(left) - (1 - pr) * self._entropy(right)
        else:
            child_uncertainty = pr * \
                self._gini_impurity(left) - (1 - pr) * self._gini_impurity(right)

        info_gain_value = parent_uncertainty - child_uncertainty
        return info_gain_value

    def _find_best_split(self, X, y):

        max_gain = -1
        best_split_question = None

        if self.criteria == "entropy":
            parent_uncertainty = self._entropy(y)
        else:
            parent_uncertainty = self._gini_impurity(y)

        m_samples, n_labels = X.shape

        for col_index in range(n_labels):  # iterate over feature columns
            # get unique values from the feature
            unique_values = np.unique(X[:, col_index])
            for val in unique_values:  # check for every value and find maximum info gain

                ques = Question(
                    column_index=col_index,
                    value=val,
                    header=self._feature_names[col_index]
                )

                t_idx, f_idx = self._partition(X, ques)
                # if it does not split the data
                # skip it
                if len(t_idx) == 0 or len(f_idx) == 0:
                    continue

                true_y = y[t_idx, :]
                false_y = y[f_idx, :]

                gain = self._info_gain(true_y, false_y, parent_uncertainty)
                if gain > max_gain:
                    max_gain, best_split_question = gain, ques

        return max_gain, best_split_question, parent_uncertainty

    def _build_tree(self, X, y, depth=0):
        m_samples, n_labels = X.shape
        if (depth > self.max_depth or n_labels == 1 or m_samples < self.min_samples_split):
            return Node(leaf_value=self._count_dict(y))

        gain, ques, uncertainty = self._find_best_split(X, y)

        if gain == 0:
            return Node(leaf_value=self._count_dict(y))

        t_idx, f_idx = self._partition(X, ques)
        true_branch = self._build_tree(X[t_idx, :], y[t_idx, :], depth + 1)
        false_branch = self._build_tree(X[f_idx, :], y[f_idx, :], depth + 1)
        return Node(
            question=ques,
            true_branch=true_branch,
            false_branch=false_branch,
            uncertainty=uncertainty
        )

    def train(self, X, y, feature_name=None, target_name=None):

        X = np.array(X, dtype='O') if not isinstance(X, (np.ndarray)) else X
        y = np.array(y, dtype='O') if not isinstance(y, (np.ndarray)) else y

        self._X = X.reshape(-1, 1) if len(X.shape) == 1 else X
        self._y = y.reshape(-1, 1) if len(y.shape) == 1 else y

        self._feature_names = feature_name or [
            f"C_{i}" for i in range(self._X.shape[1])]
        self._target_name = target_name or ['target']

        self._tree = self._build_tree(
            X=self._X,
            y=self._y
        )

    def print_tree(self, node=None, spacing="|-"):

        node = node or self._tree

        if node._is_leaf_node:
            print(spacing, " Predict :", node.leaf_value)
            return

        # Print the question at this node
        print(spacing + str(node.question) +
              " | uncertainty :" + str(node.uncertainty))

        # Call this function recursively on the true branch
        print(spacing + '--> True:')
        self.print_tree(node.true_branch, "  " + spacing + "-")

        # Call this function recursively on the false branch
        print(spacing + '--> False:')
        self.print_tree(node.false_branch, "  " + spacing + "-")

    def _classification(self, row, node):

        if node._is_leaf_node:
            return node.leaf_value

        if node.question.match(row):
            return self._classification(row, node.true_branch)
        else:
            return self._classification(row, node.false_branch)

    def _print_leaf_probability(self, results):
        total = sum(results.values())
        probs = {}
        for key in results:
            probs[key] = (results[key] / total) * 100
        return probs

    def predict(self, X):
        if isinstance(X, (np.ndarray, list)):
            X = np.array(X, dtype='O') if not isinstance(X, (np.ndarray)) else X

            if len(X.shape) == 1:
                return self._classification(row=X, node=self._tree)
            else:
                leaf_value = []
                for row in X:
                    leaf_value.append(self._classification(row=row, node=self._tree))
                return np.array(leaf_value, dtype='O')
        else:
            raise ValueError("X should be list or numpy array")

    def predict_probability(self, X):

        if isinstance(X, (np.ndarray, list)):
            X = np.array(X, dtype='O') if not isinstance(X, (np.ndarray)) else X

            if len(X.shape) == 1:
                return self._print_leaf_probability(self._classification(row=X, node=self._tree))
            else:
                leaf_value = []
                for row in X:
                    leaf_value.append(self._print_leaf_probability(
                        self._classification(row=row, node=self._tree)))
                return np.array(leaf_value, dtype='O')
        else:
            raise ValueError("X should be list or numpy array")


## Testing with Data

In [149]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import pandas as pd

dataset = load_iris()

X = dataset.data
y = dataset.target
feature_name = dataset.feature_names
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,test_size=0.10)

dt = DecisionTreeClassifier(min_samples_split=10)
dt.train(X=X_train,y=y_train,feature_name=feature_name)

dt.print_tree()

|-Is sepal length (cm) >= 7.9 ? | uncertainty :0.6663374485596707
|---> True:
  |--  Predict : {2: 1}
|---> False:
  |--Is sepal width (cm) >= 4.4 ? | uncertainty :0.6665181554912007
  |----> True:
    |---  Predict : {0: 1}
  |----> False:
    |---Is sepal width (cm) >= 4.2 ? | uncertainty :0.666402849228334
    |-----> True:
      |----  Predict : {0: 1}
    |-----> False:
      |----Is sepal width (cm) >= 4.1 ? | uncertainty :0.6662075298438934
      |------> True:
        |-----  Predict : {0: 1}
      |------> False:
        |-----Is sepal width (cm) >= 4.0 ? | uncertainty :0.6659285589417866
        |-------> True:
          |------  Predict : {0: 1}
        |-------> False:
          |------Is petal length (cm) >= 6.7 ? | uncertainty :0.6655621301775148
          |--------> True:
            |-------  Predict : {2: 2}
          |--------> False:
            |-------Is sepal length (cm) >= 7.7 ? | uncertainty :0.666015625
            |---------> True:
              |--------  Pre

In [150]:
y_pred = dt.predict_probability(X_test)
pd.DataFrame(np.vstack((y_pred,y_test)).T,columns=['predicted','original'])

,predicted,original
0,{1: 100.0},1
1,{0: 100.0},0
2,{2: 100.0},2
3,{1: 100.0},1
4,{2: 100.0},1
5,{0: 100.0},0
6,{1: 100.0},1
7,"{1: 50.0, 2: 50.0}",2
8,"{1: 50.0, 2: 50.0}",1
9,{1: 100.0},1


## Regression Tree

# Random Forest Algorithm